In [136]:
import numpy as np
import matplotlib.pyplot as plt


In [137]:
def A_dh(theta, d, a, alpha, radians = False):
    return np.array([
        [np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
        [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
        [0, np.sin(alpha), np.cos(alpha), d],
        [0, 0, 0, 1]
    ])
    
def A_trans(x, y, z):
    return np.array([
        [1, 0, 0, x],
        [0, 1, 0, y],
        [0, 0, 1, z],
        [0, 0, 0, 1]
    ])

In [138]:
dh_conv = {
    1 : {'theta': 0, 'd': 50/1000, 'a': 0, 'alpha': np.pi/2, 'type': 'revolute'},
    2 : {'theta': 0, 'd': 0, 'a': 93/1000, 'alpha': 0, 'type': 'revolute'},
    3 : {'theta': 0, 'd': 0, 'a': 93/1000, 'alpha': 0, 'type': 'revolute'},
    4 : {'theta': 0, 'd': 0, 'a': 50/1000, 'alpha': 0, 'type': 'revolute'},
    5: {'x' : -15/1000, 'y' : 45/1000, 'z' : 0, 'type': 'transformation'}
}



In [142]:
def A(phi, id):

    theta_01, d_01, a_01, alpha_01, type_01 = dh_conv[1].values()
    theta_12, d_12, a_12, alpha_12, type_12 = dh_conv[2].values()
    theta_23, d_23, a_23, alpha_23, type_23 = dh_conv[3].values()
    theta_34, d_34, a_34, alpha_34, type_34 = dh_conv[4].values()
    x_45, y_45, z_45, type_45 = dh_conv[5].values()

    _A = {
        "01": A_dh(phi, d_01, a_01, alpha_01),
        "12": A_dh(phi, d_12, a_12, alpha_12),
        "23": A_dh(phi, d_23, a_23, alpha_23),
        "34": A_dh(phi, d_34, a_34, alpha_34),
        "45": A_trans(x_45, y_45, z_45),
    }
    return _A[id]


def T(theta_1, theta_2, theta_3, theta_4, id):
    _T = {
        "00": np.eye(4),
        "01": A(theta_1, "01"),
        "02": np.dot(A(theta_1, "01"), A(theta_2, "12")),
        "03": A(theta_1, "01") @ A(theta_2, "12") @ A(theta_3, "23"),
        "04": A(theta_1, "01") @ A(theta_2, "12") @ A(theta_3, "23") @ A(theta_4, "34"),
        "05": A(0, "45") @ A(theta_1, "01") @ A(theta_2, "12") @ A(theta_3, "23") @ A(theta_4, "34"),
    }
    
    return _T[id]
    
    

In [143]:
def inv_kinematics(x,y,z,phi, dh_conv):
    """
    Hardcoded inverse kinematics for the robot arm
    """
    
    theta1 = np.atan2(y, x)
    
    a2 = dh_conv[2]['a']
    a3 = dh_conv[3]['a']
    a4 = dh_conv[4]['a']
    
    d1 = dh_conv[1]['d']
    
    r = np.sqrt(x**2 + y**2)
    r2 = np.cos(phi)*a4
    
    #Given r = r1 + r2
    r1 = r - r2
    
    z1 = d1
    z3 = np.sin(phi)*a4
    
    #Given z = z1 + z2 + z3
    z2 = z - z1 - z3
    
    c23 = np.sqrt(z2**2 + r1**2)
    
    c3 = ((c23**2) - (a2**2) - (a3**2)) / (2*a2*a3)
    
    
    D = (c23**2 - a2**2 - a3**2)/(-2*a2*a3)
    theta3 = -(np.atan2(np.sqrt(1-D**2), D) - np.pi)

    
    # #two solutions for theta3
    # print("c3", c3)
    
    # theta3_1 = atan2(-sqrt(1 - c3**2), c3)
    # theta3_2 = atan2(sqrt(1 - c3**2), c3)
    
    
    # print("theta3_1", theta3_1)
    # print("theta3_2", theta3_2)
    
    # theta3 = theta3_1
    
    theta2 = np.atan2(z2, r1) - np.atan2(a3*np.sin(theta3), a2 + a3*np.cos(theta3))
    
    theta4 = phi - theta2 - theta3
    
    
    return theta1, theta2, theta3, theta4

inv_kinematics(-0.032,0.15, 0.12, 0, dh_conv)


(np.float64(1.7809789134570662),
 np.float64(-0.23973021695178431),
 np.float64(1.6699026205990042),
 np.float64(-1.4301724036472199))

In [144]:
def point_on_circle(phi: float, radius: float = 32/1000):
    centre = np.array([150/1000, 0, 120/1000])
    return centre + radius * np.array([0, np.cos(phi), np.sin(phi)])

def jacobian_inner(t, joint, o_n):
    joint_type = joint['type']
    z = t[:3,2]
    o = t[:3,3]
    
    
    if joint_type == 'revolute':
        j = np.cross(z, o_n - o)
        return np.hstack([j, z]).T
    else:
        return np.hstack([z, np.zeros(3,1)]).T
    

def jacobian(T_, joints):
    o_n = T_[-1][:3,3]
    J = np.zeros((6, len(joints)))
    for idx, joint in enumerate(joints):
        t = T_[idx]
        j = jacobian_inner(t, joint, o_n)
        
        
        J[:, idx] = j
        
    return J


In [166]:
def jacobian_circle_angle(phi, end_angle, dh_conv):

    _thetas = inv_kinematics(*point_on_circle(phi), end_angle, dh_conv)

    _T = [T(*_thetas, id) for id in ["00", "01", "02", "03", "04", "05"]]
    _joints = list(dh_conv.values())
    J = jacobian(_T[:5], _joints[:4])
    return J
    np.round(J_end, 3)

In [167]:

def joint_velocity(phi, velocity, end_angle, dh_conv):
    M = jacobian_circle_angle(phi, end_angle, dh_conv)[0:3, :] 

    N = np.linalg.pinv(M)
    
    q = N @ velocity
    
    return q
    


In [ ]:
def interpolation_polynomial(dh_conv, start_angle, end_angle, t_in, t_end, start_vel, end_vel):
    """Computes polynomial interpolation"""
    point_start = point_on_circle(start_angle)
    point_end = point_on_circle(end_angle)
    q_in = inv_kinematics(*point_start, 0, dh_conv)
    q_end = inv_kinematics(*point_end, 0, dh_conv)
    
    A_list = []
    points = []
    # print(t_in, t_end)
    for i in range(4):
        b = Matrix([q_in[i], start_vel[i], 0, q_end[i], end_vel[i], 0])
        
        M = Matrix([
            [1, t_in, t_in**2, t_in**3, t_in**4, t_in**5],
            [0, 1, 2*t_in, 3*t_in**2, 4*t_in**3, 5*t_in**4],
            [0, 0, 2, 6*t_in, 12*t_in**2, 20*t_in**3],
            [1, t_end, t_end**2, t_end**3, t_end**4, t_end**5],
            [0, 1, 2*t_end, 3*t_end**2, 4*t_end**3, 5*t_end**4],
            [0, 0, 2, 6*t_end, 12*t_end**2, 20*t_end**3]
        ])

        M_inv = M.inv()

        A = M_inv * b
        
        A_list.append(A)
        points.append(point_start)
        
        
    return A_list, points

array([-2.00000000e-02, -3.22981556e-17,  8.26176463e-17, -6.92353575e-17])